# ALUMNO: Ezequiel Scordamaglia

<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [31]:
def get_vocabulario(corpus):
    terminos_unidos = np.concatenate([frase.split() for frase in corpus])
    vocabulario = np.unique(terminos_unidos)
    return vocabulario

print(get_vocabulario(corpus))

['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [34]:
def vectorizar_ohe(corpus):

    vocabulario = get_vocabulario(corpus)
    corpus_vectorizado = np.zeros((len(corpus), len(vocabulario)))
    for documento in corpus:
        for termino in documento.split():
            corpus_vectorizado[corpus == documento, vocabulario == termino] = 1
    return corpus_vectorizado

vectorizar_ohe(corpus)

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [33]:
def vectorizar_vf(corpus):

    vocabulario = get_vocabulario(corpus)
    corpus_vectorizado = np.zeros((len(corpus), len(vocabulario)))
    for documento in corpus:
        for termino in documento.split():
            corpus_vectorizado[corpus == documento, vocabulario == termino] += 1
    return corpus_vectorizado

vectorizar_vf(corpus)

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 2., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [36]:
def vectorizar_tfidf(corpus):
    vocabulario = get_vocabulario(corpus)

    # Calculo del IDF de cada palabra
    idf = np.zeros(len(vocabulario))
    for i, termino in enumerate(vocabulario):
        idf[i] = np.log(len(corpus) / np.sum([termino in documento for documento in corpus]))

    corpus_vectorizado = np.zeros((len(corpus), len(vocabulario)))
    for documento in corpus:
        for termino in documento.split():
            corpus_vectorizado[corpus == documento, vocabulario == termino] += 1
        
        corpus_vectorizado = corpus_vectorizado * idf

    return corpus_vectorizado

vectorizar_tfidf(corpus)

array([[0.        , 0.06665926, 0.        , 0.        , 0.        ,
        0.06665926, 0.        , 0.        , 1.32596896],
       [1.20694896, 0.16440195, 1.20694896, 0.        , 0.        ,
        0.16440195, 0.32880391, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.09861229,
        0.        , 0.40546511, 1.09861229, 0.        ]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [46]:
def comparacion_similaridad(corpus, idx, vectorizacion='ohe'):
    """
    Función que compara la similitud entre un documento y el resto del corpus.
    
    :corpus - Arreglo de documentos.
    :idx - Indice del documento a comparar.
    :vectorizacion - Tipo de vectorización a utilizar. Puede ser 'ohe', 'vf' o 'tfidf'.
    """

    # Vectorizar corpus
    if vectorizacion == 'ohe':
        corpus_vectorizado = vectorizar_ohe(corpus)
    elif vectorizacion == 'vf':
        corpus_vectorizado = vectorizar_vf(corpus)
    elif vectorizacion == 'tfidf':
        corpus_vectorizado = vectorizar_tfidf(corpus)

    vector_similitudes = np.zeros(len(corpus))
    for i, documento in enumerate(corpus_vectorizado):
        vector_similitudes[i] = cosine_similarity(corpus_vectorizado[idx], documento)
    
    # Ordenar corpus por similitud, agregar columna con similitudes
    corpus_ordenado = corpus[vector_similitudes.argsort()[::-1]]
    corpus_ordenado = np.c_[corpus_ordenado, vector_similitudes[vector_similitudes.argsort()[::-1]]]
    return corpus_ordenado   

In [54]:
print("\nUsando One Hot Encoding")
for i in range(len(corpus)):
    print('Documento: ', corpus[i])
    documentos_similares = comparacion_similaridad(corpus, i, 'ohe')
    for documento in documentos_similares:
        print(documento)

print("\nUsando Vector de Frecuencias")
for i in range(len(corpus)):
    print('Documento: ', corpus[i])
    documentos_similares = comparacion_similaridad(corpus, i, 'vf')
    for documento in documentos_similares:
        print(documento)

print("\nUsando TFIDF")
for i in range(len(corpus)):
    print('Documento: ', corpus[i])
    documentos_similares = comparacion_similaridad(corpus, i, 'tfidf')
    for documento in documentos_similares:
        print(documento)


Usando One Hot Encoding
Documento:  que dia es hoy
['que dia es hoy' '1.0']
['martes el dia de hoy es martes' '0.6123724356957946']
['martes muchas gracias' '0.0']
Documento:  martes el dia de hoy es martes
['martes el dia de hoy es martes' '1.0000000000000002']
['que dia es hoy' '0.6123724356957946']
['martes muchas gracias' '0.23570226039551587']
Documento:  martes muchas gracias
['martes muchas gracias' '1.0000000000000002']
['martes el dia de hoy es martes' '0.23570226039551587']
['que dia es hoy' '0.0']

Usando Vector de Frecuencias
Documento:  que dia es hoy
['que dia es hoy' '1.0']
['martes el dia de hoy es martes' '0.5']
['martes muchas gracias' '0.0']
Documento:  martes el dia de hoy es martes
['martes el dia de hoy es martes' '1.0']
['que dia es hoy' '0.5']
['martes muchas gracias' '0.3849001794597505']
Documento:  martes muchas gracias
['martes muchas gracias' '1.0000000000000002']
['martes el dia de hoy es martes' '0.3849001794597505']
['que dia es hoy' '0.0']

Usando TFID